<a href="https://colab.research.google.com/github/cezarviana/fake-news-no-more/blob/main/test2_fake_news_no_more_(et_varginha).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai


# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk


from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Priorize fontes jornalísticas reconhecidas e com boa reputação.
        Selecione no máximo 5 lançamentos que demonstrem relevância (baseada na cobertura e qualidade da fonte) e sejam os mais atuais possíveis.
        Se o tópico não possuir 5 notícias a seu respeito, apresente somente as encontradas. Sem adicionar outras notícias com termos parecidos e que não tenham relação direta com o tópico.
        Se o tópico gerar pouca cobertura noticiosa ou reações limitadas, sinalize essa baixa relevância como um possível indicativo para considerar outros tópicos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca de notícias no Google Search sobre o tópico",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


################################################
# --- Agente 2: Verificador de Fontes --- #
################################################
def agente_verificador_fontes(topico, lancamentos_buscados):
    verificador_fontes = Agent(
        name="agente_verificador_fontes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Fontes #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de fontes, especialista em fact-checking.
        Para cada fonte principal identificada:
        - Determine se o site ou canal é conhecido e geralmente considerado confiável.
        - Localize e examine a seção 'Sobre nós' (ou equivalente) para entender a missão, equipe e possíveis vieses do site.
        - Verifique se outras fontes confiáveis corroboram as informações apresentadas pela fonte principal. Liste as fontes que confirmam os achados.
        """,
        description="Agente que verifica as fontes levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fontes = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_fontes = call_agent(verificador_fontes, entrada_do_agente_verificador_fontes)
    return verificacao_fontes


################################################
# --- Agente 3: Verificador de Conteúdo --- #
################################################
def agente_verificador_conteudo(topico, lancamentos_buscados):
    verificador_conteudo = Agent(
        name="agente_verificador_conteudo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Conteúdo #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de conteúdo, especialista em fact-checking.
        Examine o conteúdo das notícias e informações relacionadas ao tópico.
        Linguagem e Estilo:
        - Avalie se há uso de sofismos ou outras técnicas de persuasão manipuladoras.
        - Identifique erros de ortografia e gramática que possam indicar falta de profissionalismo ou revisão.
        - Distinga claramente entre fatos apresentados e opiniões, verificando se as opiniões são devidamente atribuídas.
        Contexto e Evidências:
        - Verifique se a data da informação é relevante para o contexto atual.
        - Se houver imagens, utilize a busca reversa (Google Imagens) para verificar sua autenticidade e se foram usadas em outros contextos enganosos.
        Coerência:
        - Avalie a lógica interna do conteúdo e sua coerência com informações de outras fontes.
        """,
        description="Agente que verifica o conteúdo das notícias levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_conteudo = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_conteudo = call_agent(verificador_conteudo, entrada_do_agente_verificador_conteudo)
    return verificacao_conteudo


##########################################
# --- Agente 4: Agência de Fact-Checking --- #
##########################################
def agente_verificador_fatos(topico, data_de_hoje):
    verificador_fatos = Agent(
        name="agente_verificador_fatos",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua tarefa é verificar se o tópico/afirmação já foi alvo de checagem por agências de fact-checking confiáveis, através da  busca google (google_search), priorizando as verificações mais recentes sobre o tópico.
        As agências de fact-checking a serem consultadas de acordo com o tópico serão as indicadas abaixo:
        - No Brasil:
          - Aos Fatos;
          - Lupa;
          - UOL Confere;
          - Estadão Verifica;
          - Fato ou Fake (G1);
          - Boatos.org;
          - Agência Pública - Truco no Congresso;
          - Comprova;
          - E-farsas;
          - É isso Mesmo? (O Globo);
          - Portal EBC - Checagem;
        - Internacionais com atuação ou relevância no Brasil:
          - AFP Fact Check;
          - Reuters Fact Check;
          - Snopes;
          - PolitiFact;
          - FactCheck.org;
        - Organizações e Redes Internacionais:
          - International Fact-Checking Network (IFCN);
          - European Fact-Checking Standards Network (EFCSN);
          - Duke Reporters' Lab;
        Informe se o tópico foi encontrado em alguma das agências de fact-checking e qual foi a conclusão dessas agências (verdadeiro, falso, enganoso, etc.).
        Se encontrado, cite a fonte da agência de fact-checking e um breve resumo da sua análise.
        """,
        description="Agente que verifica o que agências de fact-checking dizem a respeito do tópico",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fatos = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    verificacao_fatos = call_agent(verificador_fatos, entrada_do_agente_verificador_fatos)
    return verificacao_fatos


###############################################
# --- Agente 5: Organizador do Resultado da Verificação --- #
################################################
def agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos):
    organizador = Agent(
        name="agente_organizador_resultado",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de Resultados #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é organizar os resultados com base nas análises dos outros agentes sobre o tópico, e determinar a veracidade da informação.
        Apresente um veredicto claro: Verdadeiro, Falso, Enganoso, Insustentável, etc.
        Justifique sua conclusão de forma concisa, utilizando as evidências e os resultados fornecidos pelos outros agentes (agente_buscador, agente_verificador_fontes, agente_verificador_conteudo e agente_verificador_fatos).
        Liste as fontes mais relevantes (sites de notícias confiáveis, agências de fact-checking) que sustentam sua conclusão.
        """,
        description="Agente que organiza os resultados da verificação",
        tools=[google_search]
    )

    entrada_do_agente_organizador_resultado = f"Tópico:{topico}\nLançamentos buscados:{lancamentos_buscados}\nVerificação fontes:{verificacao_fontes}\nVerificação conteúdo:{verificacao_conteudo}\nVerificação fatos:{verificacao_fatos}"
    # Executa o agente
    resultado = call_agent(organizador,  entrada_do_agente_organizador_resultado)
    return resultado

In [5]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos pesquisar sobre a veracidade a respeito de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    verificacao_fontes = agente_verificador_fontes(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---\n")
    display(to_markdown(verificacao_fontes))
    print("--------------------------------------------------------------")

    verificacao_conteudo = agente_verificador_conteudo(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---\n")
    display(to_markdown(verificacao_conteudo))
    print("--------------------------------------------------------------")

    verificacao_fatos = agente_verificador_fatos(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---\n")
    display(to_markdown(verificacao_fatos))
    print("--------------------------------------------------------------")

    resultado = agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos)
    print("\n--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: Et de Varginha
Maravilha! Vamos pesquisar sobre a veracidade a respeito de Et de Varginha

--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---



> Olá! Para ajudar na verificação de notícias sobre o ET de Varginha, vou pesquisar no Google as notícias mais recentes e relevantes sobre o tema.
> 
> 
> Com base nas pesquisas realizadas, selecionei as notícias mais recentes sobre o ET de Varginha:
> 
> 1.  **Aniversário do Caso e Turismo Ufológico:** Em 20 de janeiro de 2025, o caso do ET de Varginha completou 29 anos, mantendo a cidade como um destino de turismo ufológico. A cidade de Varginha se tornou mundialmente conhecida como a "cidade do ET" devido aos eventos de janeiro de 1996, que envolveram avistamentos de OVNIs e possíveis seres não humanos.
> 2.  **Registro como Patrimônio Cultural Imaterial:** A prefeitura de Varginha está realizando estudos para registrar o "Caso ET de Varginha" como Patrimônio Cultural Imaterial local. Essa iniciativa visa promover ações de salvaguarda com foco na educação patrimonial, incentivando estudos da astronomia, conhecimentos espaciais e ufologia.
> 3.  **Aquisição de Terreno:** A prefeitura de Varginha está negociando a aquisição do terreno onde ocorreu o suposto avistamento do ET. A secretária municipal de Turismo, Rosana Carvalho, espera que o processo seja concluído em breve, com planos de investir mais de R$ 5 milhões para tornar o local mais interativo.
> 4.  **Documentário "Moment of Contact":** O cineasta norte-americano James Fox lançou o documentário "Moment of Contact – O Caso Varginha", que mantém a história viva e fomenta discussões e novos relatos sobre o caso. O documentário foi exibido gratuitamente para os moradores de Varginha.
> 5.  **Memorial do ET:** O Memorial do ET em Varginha recebeu mais de 20 mil visitantes desde a sua inauguração, com cerca de 300 a 400 pessoas visitando o local a cada fim de semana. A arquitetura do memorial imita um disco voador.
> 
> Esses lançamentos demonstram a contínua relevância do caso do ET de Varginha, tanto para o turismo local quanto para a cultura e memória da cidade.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---



> Para verificar a veracidade das notícias sobre o ET de Varginha, vou analisar as fontes e procurar por corroborações em fontes confiáveis.
> 
> **1. Aniversário do Caso e Turismo Ufológico:**
> 
> *   **Fonte:** A notícia menciona o aniversário de 29 anos do caso em 20 de janeiro de 2025 e o turismo ufológico.
> *   **Verificação:** Varginha é conhecida como a "cidade do ET" devido aos eventos de 1996. A data do caso é amplamente divulgada, e o turismo ufológico é um fato conhecido na região.
> 
> **2. Registro como Patrimônio Cultural Imaterial:**
> 
> *   **Fonte:** A prefeitura de Varginha está estudando registrar o "Caso ET de Varginha" como Patrimônio Cultural Imaterial local.
> *   **Verificação:** Essa informação pode ser confirmada através de notícias da prefeitura de Varginha ou veículos de comunicação locais.
> 
> **3. Aquisição de Terreno:**
> 
> *   **Fonte:** A prefeitura de Varginha está negociando a aquisição do terreno do suposto avistamento.
> *   **Verificação:** Similarmente, essa informação deve ser confirmada por fontes oficiais da prefeitura ou notícias locais.
> 
> **4. Documentário "Moment of Contact":**
> 
> *   **Fonte:** O documentário "Moment of Contact – O Caso Varginha" foi lançado pelo cineasta James Fox.
> *   **Verificação:** A existência do documentário e o envolvimento de James Fox podem ser verificados através de sites de cinema, entrevistas com o cineasta e plataformas de streaming.
> 
> **5. Memorial do ET:**
> 
> *   **Fonte:** O Memorial do ET em Varginha recebeu mais de 20 mil visitantes desde a sua inauguração.
> *   **Verificação:** A existência do memorial e o número de visitantes podem ser confirmados através de informações da prefeitura de Varginha, notícias locais e relatos de visitantes.
> 
> Para cada um desses pontos, seria ideal buscar fontes primárias (como o site da prefeitura de Varginha) ou reportagens de veículos de imprensa reconhecidos para confirmar as informações.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---



> O caso do ET de Varginha tem sido alvo de diversas análises e investigações ao longo dos anos. Agências de fact-checking e veículos de comunicação abordaram o tema, buscando esclarecer os fatos e separar a realidade das teorias da conspiração.
> 
> **Análise de Agências de Fact-Checking e Veículos de Mídia:**
> 
> *   **Boatos.org:** Em 2024, o Boatos.org desmentiu a informação de que um vídeo sobre o ET de Varginha teria vazado na internet, classificando-o como falso. A análise apontou que o vídeo surgiu muitos anos após o ocorrido, sem identificação do autor, e com perda de qualidade nas imagens do suposto ET.
> *   **E-farsas:** O site E-farsas publicou um episódio do "Fake em Nóis" revelando "toda a verdade" sobre o caso do ET de Varginha.
> *   **UOL Confere:** O UOL Confere destaca o caso do ET de Varginha como o mais famoso caso de óvni do Brasil, mencionando o suposto avistamento por um casal de agricultores e o encontro de três adolescentes com uma criatura misteriosa.
> *   **Fato ou Fake (G1):** O programa "Investigação Criminal Paranormal" do canal Fato ou Fake no YouTube, abordou a questão "O E.T. DE VARGINHA EXISTE?", explorando os mistérios e acontecimentos que marcaram a cidade em 1996.
> 
> **Outras Abordagens e Perspectivas:**
> 
> *   **Teorias da Conspiração vs. Realidade:** A história do ET de Varginha é permeada por teorias da conspiração, incluindo relatos de captura e transporte do suposto ET para Campinas, suposto envolvimento da CIA e acobertamento militar. No entanto, autoridades locais e federais negam as alegações, e ufólogos apontam a falta de evidências físicas.
> *   **Ceticismo e Falta de Provas:** Ufólogos como Kevin Randle e Ubirajara Rodrigues reconhecem a ausência de evidências concretas que sustentem o caso.
> *   **Explicações Racionais:** Algumas explicações racionais para o caso incluem a confusão com um morador local com problemas mentais, conhecido como "Mudinho", ou a identificação errônea de animais silvestres.
> *   **Turismo Ufológico e Patrimônio Imaterial:** Apesar da falta de comprovação, o caso do ET de Varginha impulsionou o turismo na cidade, que se auto-intitula "cidade do ET". Em 2023, a prefeitura iniciou estudos para registrar o caso como Patrimônio Cultural Imaterial local, visando a educação patrimonial e o estudo da ufologia.
> *   **Documentários e Filmes:** O caso também foi retratado em documentários como "Moment of Contact", que apresenta entrevistas com testemunhas e novas perspectivas sobre os eventos de 1996.
> 
> **Em resumo:** Embora o caso do ET de Varginha continue a gerar fascínio e discussões, não há evidências concretas que confirmem a aparição ou captura de um ser extraterrestre. As análises de agências de fact-checking e as investigações realizadas ao longo dos anos apontam para a necessidade de um olhar crítico sobre as informações divulgadas e a importância de separar os fatos das especulações.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---



> Com base na análise das informações e verificações realizadas, o tópico sobre o ET de Varginha é considerado **insustentável**.
> 
> **Justificativa:**
> 
> *   **Falta de Evidências Concretas:** Apesar da fama do caso e do turismo ufológico associado, não existem evidências físicas ou documentais que comprovem a aparição ou captura de um ser extraterrestre em Varginha.
> *   **Teorias da Conspiração:** Muitas alegações são baseadas em teorias da conspiração, sem respaldo em fatos verificáveis.
> *   **Explicações Racionais:** Existem explicações alternativas para os relatos, como confusão com pessoas ou animais.
> *   **Análise de Fact-Checking:** Agências de fact-checking têm desmentido informações falsas relacionadas ao caso, como a divulgação de vídeos fraudulentos.
> 
> **Fontes Relevantes:**
> 
> *   Boatos.org
> *   E-farsas
> *   UOL Confere
> *   Fato ou Fake (G1)
> 


--------------------------------------------------------------
